In [ ]:
import os
import sys
import math
import mat73
import warnings
import numpy as np
from tqdm import trange
import matplotlib.pyplot as plt
from scipy.optimize import nnls
from sklearn.decomposition import NMF
from scipy.io import loadmat, savemat
from matplotlib.patches import Rectangle
from sklearn.decomposition import FastICA
from matplotlib.colors import ListedColormap
from utils import *

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
abs_coeff, skin_dict, water_dict = {}, {}, {}
wave_abs = np.load('../data/hbo2hbchpr_57.npy')
plt3d = loadmat('../data/3DPlot.mat')
X, Y = plt3d['x'], plt3d['y']
abs_dict = {}

for idx, wave in enumerate(np.arange(700, 981, 5)):
    abs_coeff[wave] = wave_abs[idx]
    skin = (0.244 + 85.3 * math.exp(-(wave - 154)/66.2)) / 100
    melanin = (1.70 * 1e12 * math.pow(wave, -3.48)) / 100
    skin_dict[wave] = np.array([skin, melanin])
    abs_dict[int(wave)] = list(np.append(np.append(wave_abs[idx], skin), melanin))

wave_list = [750, 800, 850]
coeffs = np.vstack([abs_coeff[wave] for wave in wave_list])
fcoeffs = np.vstack([np.append(abs_coeff[wave], skin_dict[wave], axis = 0) for wave in wave_list])
norm_coeffs = np.array([fcoeffs.T[idx] / max(coeffs.T[idx]) for idx in range(coeffs.shape[1])]).T
legend = ['HbO', 'Hb', 'Cholesterol']
melorder = ['HbO2', 'Hb', 'Cholesterol', 'Melanin']
skinorder = ['HbO2', 'Hb', 'Cholesterol', 'Skin Baseline']
hbhbo2fat = np.copy(coeffs)[:, [0, 1, 2]]
hbhbo2fatskin = np.copy(fcoeffs[:, [0, 1, 2, 4]])
hbhbo2fatmelanin = np.copy(fcoeffs[:, [0, 1, 2, 5]])

mixed_coeffs = []
for idx in [760, 808, 915]:
    if idx == 808:
        ids = 0.69 * abs_coeff[805] + 0.31 * abs_coeff[940]
    else:
        ids = abs_coeff[idx]
    # print(f'{idx} {ids[0]:.6f} {ids[1]:.6f} {ids[2]:.6f} {ids[3]:.6f}')
    mixed_coeffs.append(ids)
mixed_coeffs = np.array(mixed_coeffs)[:, :3]

# Hypercapnia

In [ ]:
imgfile = loadmat("../../acousticx/data/beamformed/11242023-HYP-915-808-LOC2.mat")['beamformed']
waves = [915, 808]
wave_dict = {760: 0, 808: 1, 915: 2}
wave_list = [wave_dict[idx] for idx in waves]
title = [f'PRE CO2 {waves[0]}NM', f'CO2 {waves[0]}NM', f'POST CO2 {waves[0]}NM', f'PRE CO2 {waves[1]}NM', f'CO2 {waves[1]}NM', f'POST CO2 {waves[1]}NM']
plt.imshow(imgfile[0][100:, :], cmap = 'hot')
plt.colorbar()

In [ ]:
roi = [350, 550]
plt.figure(figsize = (12, 8))
for idx in range(imgfile.shape[0]):
    plt.subplot(2, 3, idx + 1)
    plt.imshow(imgfile[idx][roi[0]:roi[1], :], cmap = 'hot', extent = [0, 1, 0, 1])
    plt.colorbar()
    plt.clim([0, np.max(imgfile[:, roi[0]:roi[1], :])])
    plt.title(title[idx])

In [ ]:
title_order = ['PRE CO2', 'CO2', 'POST CO2']
order = ['HbO', 'Hb', 'Cholesterol']
imgdict, didx = {}, 0
for i in [[0, 1], [0, 2], [1, 2]]:
    for idx, j in enumerate([[0, 3], [1, 4], [2, 5]]):
        imgdict[didx] = (title_order[idx], i, j)
        didx += 1
imgidx = 1
plt.figure(figsize = (24, 12))
for idx in range(9):
    exp_unmixed = run_linear_unmixing(normalize(imgfile[imgdict[idx][2], roi[0]:roi[1], :].copy()), mixed_coeffs[wave_list][:, imgdict[idx][1]])
    for xidx in range(2):
        plt.subplot(3, 6, imgidx)
        plt.imshow(exp_unmixed[:, :, xidx], cmap = "hot", extent = [0, 1, 0, 1])
        plt.colorbar()
        plt.title(f"{imgdict[idx][0]} {order[imgdict[idx][1][xidx]]}")
        plt.clim([0, np.max(exp_unmixed)])
        imgidx += 1
plt.show()

# Time Series Analysis 

In [ ]:
def moving_average(data, window_size):
    cumsum = np.cumsum(data, dtype=float)
    cumsum[window_size:] = cumsum[window_size:] - cumsum[:-window_size]
    return cumsum[window_size - 1:] / window_size

In [ ]:
pafile = loadmat("../../acousticx/data/beamformed/11062023-ft_760_even_915_odd_co2_30_90_30_OBP_Laser_PA_1477.mat")['beamformed']
plt.figure(figsize = (15, 10))
roi_size = 7
rois = [[559, 41], [556, 51], [495, 45], [471, 51]]
for idx in range(len(rois)):
    plt.subplot(2, 2, idx + 1)
    mean = np.mean(pafile[:, rois[idx][0]:rois[idx][0] + roi_size, rois[idx][1]:rois[idx][1] + roi_size], axis = (1, 2))
    mavg = moving_average(mean, 5)
    plt.plot(list(range(pafile.shape[0])), mean, "b")
    plt.plot(list(range(mavg.shape[0])), mavg, 'r')
    plt.title(f'ROI - {idx + 1}')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.imshow(pafile[10, :, :], cmap='hot')
for ridx in range(len(rois)):
    rect = Rectangle((rois[ridx][1], rois[ridx][0]), roi_size, roi_size, linewidth = 1, edgecolor='white', facecolor='none')
    ax.add_patch(rect)
plt.show()